In [1]:
import os
import sys
import json
import time
import logging

# Crear un manejador personalizado
class CustomFormatter(logging.Formatter):
    def format(self, record):
        # Usar el método getMessage() y agregar la hora formateada
        formatted_time = self.formatTime(record, datefmt='%Y-%m-%d %H:%M:%S')
        return f"{formatted_time} - {record.getMessage()}"

# Crear un logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Crear un manejador que imprime en stdout
handler = logging.StreamHandler(stream=sys.stdout)
formatter = CustomFormatter()  # Usar el CustomFormatter
handler.setFormatter(formatter)

# Limpiar manejadores previos si los hay
if logger.hasHandlers():
    logger.handlers.clear()

# Agregar el nuevo manejador
logger.addHandler(handler)

install_path = os.getenv("INSTALL_PATH")
print(install_path)# como no me lo pilla pues lo pongo a mano
sys.path.insert(0, "/mnt/netapp1/Store_CESGA/home/cesga/mlosada/api/api-simulator/installation")
sys.path.insert(0, "/mnt/netapp1/Store_CESGA/home/cesga/mlosada/api/api-simulator/python")

/mnt/netapp1/Store_CESGA/home/cesga/mlosada/api/api-simulator/installation


In [2]:
from qpu import QPU, getQPUs
from qjob import gather

In [3]:
qpus = getQPUs()
print("QPUs disponibles:")
for q in qpus:
    print(f"QPU {q.id_}, backend {q.backend.name}.")

QPUs disponibles:
QPU 0, backend FakeQmio.


In [ ]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(2,2)

In [156]:
with open("circuit_15qubits_10layers.json", "r") as file:
    circuit = json.load(file)

Mandando los circuitos así funciona perfecto.

In [158]:
qjobs = []

tick = time.time()
for q in qpus:
    qjobs.append(q.run(circ = circuit, shots=222))
    logger.info(f"Enviado a QPU {q.id_}")

logger.info("Dentro gather: ")

resultados = gather(qjobs)

tack = time.time()

print("Terminó en :", tack-tick, " s")

for i,r in enumerate(resultados):
    tiempo = r.get_dict()["results"][0]["time_taken"]
    print(f"For QPU {i}: {tiempo} s")

2025-01-23 10:50:14 - Enviado a QPU 0
2025-01-23 10:50:14 - Enviado a QPU 1
2025-01-23 10:50:14 - Enviado a QPU 2
2025-01-23 10:50:14 - Enviado a QPU 3
2025-01-23 10:50:14 - Enviado a QPU 4
2025-01-23 10:50:14 - Dentro gather: 
Terminó en : 74.73958396911621  s
For QPU 0: 74.677205631 s
For QPU 1: 74.617190847 s
For QPU 2: 74.623033808 s
For QPU 3: 74.652475401 s
For QPU 4: 74.640246529 s


In [272]:
resultados[0].get_counts()

{'10000101100001000000101100011': 1,
 '10000110011001000000101111101': 1,
 '10001010100011000000000010011': 1,
 '10110001001010000000011110011': 1,
 '10110001010100000000011111001': 1,
 '10110011000001000000001010011': 1,
 '10110011011101000001101111111': 1,
 '10110100010111000000111111011': 1,
 '10110101010001000000110110011': 1,
 '10110101010011000001001111011': 1,
 '10110110001000000000101001111': 1,
 '10110110010101000000101111001': 1,
 '10110110100101000001000011001': 1,
 '10111010100000000000001010101': 1,
 '10111011010001000000011111011': 1,
 '10111100110011000000111010001': 1,
 '10111110001111000001100110011': 1,
 '10111110010100000000100111001': 1,
 '10111110010100000001100111011': 1,
 '11000000100011000000010100111': 1,
 '11000000110100000000011101111': 1,
 '11000001010011000001111110101': 1,
 '11000001011100000000001110111': 1,
 '11000001110100000000011011001': 1,
 '11000001110100000000111011011': 1,
 '11000010010010000000000110001': 1,
 '11000010110000000001100110011': 1,
 

Vamos a probar mandando como `QuantumCircuit`, debería ser lo mismo ya que el paso de pasar a qc_to_json es el mismo

In [27]:
import numpy as np
from qiskit import QuantumCircuit

num_qubits = 15

layers = 10

params = []

for l in range(layers):
    params.append(np.random.uniform(0, 2 * np.pi, num_qubits))
                  
qc = QuantumCircuit(num_qubits,num_qubits)

qc.h(range(num_qubits))

for j,pa in enumerate(params):

    for i in range(num_qubits-1):
        if i%2 == 0 :
            qc.cx(i,i+1)
    
    for i,p in enumerate(pa):
        qc.rx(p, i)
        qc.ry(p,i)
    qc.measure(j,j)
    
    qc.h(range(num_qubits))

qc.measure_all()

In [50]:
from qiskit.circuit import QuantumRegister, ClassicalRegister, Instruction, Operation, Clbit
qc.cregs

[ClassicalRegister(15, 'c'), ClassicalRegister(15, 'meas')]

In [58]:
c = Clbit(ClassicalRegister(15, 'c'), 0)

c.name
    

AttributeError: 'Clbit' object has no attribute 'name'

In [65]:
qc.data[52]

CircuitInstruction(operation=Instruction(name='measure', num_qubits=1, num_clbits=1, params=[]), qubits=(Qubit(QuantumRegister(15, 'q'), 0),), clbits=(Clbit(ClassicalRegister(15, 'c'), 0),))

In [66]:
qc.data[-1]

CircuitInstruction(operation=Instruction(name='measure', num_qubits=1, num_clbits=1, params=[]), qubits=(Qubit(QuantumRegister(15, 'q'), 14),), clbits=(Clbit(ClassicalRegister(15, 'meas'), 14),))

In [20]:
from qiskit.circuit import Instruction

instruction = Instruction(name= 'h', num_qubits = 1, num_clbits = 0, params = [])
qubit = 

In [18]:
circuit['instructions'][0]

{'name': 'h', 'qubits': [0], 'params': []}

In [21]:
circuit

{'instructions': [{'name': 'h', 'qubits': [0], 'params': []},
  {'name': 'h', 'qubits': [1], 'params': []},
  {'name': 'h', 'qubits': [2], 'params': []},
  {'name': 'h', 'qubits': [3], 'params': []},
  {'name': 'h', 'qubits': [4], 'params': []},
  {'name': 'h', 'qubits': [5], 'params': []},
  {'name': 'h', 'qubits': [6], 'params': []},
  {'name': 'h', 'qubits': [7], 'params': []},
  {'name': 'h', 'qubits': [8], 'params': []},
  {'name': 'h', 'qubits': [9], 'params': []},
  {'name': 'h', 'qubits': [10], 'params': []},
  {'name': 'h', 'qubits': [11], 'params': []},
  {'name': 'h', 'qubits': [12], 'params': []},
  {'name': 'h', 'qubits': [13], 'params': []},
  {'name': 'h', 'qubits': [14], 'params': []},
  {'name': 'cx', 'qubits': [0, 1], 'params': []},
  {'name': 'cx', 'qubits': [2, 3], 'params': []},
  {'name': 'cx', 'qubits': [4, 5], 'params': []},
  {'name': 'cx', 'qubits': [6, 7], 'params': []},
  {'name': 'cx', 'qubits': [8, 9], 'params': []},
  {'name': 'cx', 'qubits': [10, 11], 'p

In [8]:
qjobs = []

tick = time.time()
for q in qpus:
    qjobs.append(q.run(circ = qc, shots=222))# mando aquiiiiiiiiiiiiiiiiiiii !!!!!!!!!!!!!!!!!!
    logger.info(f"Enviado a QPU {q.id_}")

logger.info("Dentro gather: ")

resultados = gather(qjobs)

tack = time.time()

print("Terminó en :", tack-tick, " s")

for i,r in enumerate(resultados):
    tiempo = r.get_dict()["results"][0]["time_taken"]
    print(f"For QPU {i}: {tiempo} s")

2025-01-20 10:30:59 - Enviado a QPU 0
2025-01-20 10:30:59 - Enviado a QPU 1
2025-01-20 10:30:59 - Enviado a QPU 2
2025-01-20 10:30:59 - Enviado a QPU 3
2025-01-20 10:30:59 - Enviado a QPU 4
2025-01-20 10:30:59 - Dentro gather: 
Terminó en : 74.71957111358643  s
For QPU 0: 74.647070954 s
For QPU 1: 74.631572544 s
For QPU 2: 74.641695891 s
For QPU 3: 74.597587271 s
For QPU 4: 74.608502667 s


## Tema Backend:

In [9]:
# Configuración proporcionada
config = {
    'backend_name': 'BasicAer',
    'backend_version': '0.0.1',
    'n_qubits': 32,
    'basis_gates': ['u1', 'u2', 'u3', 'x', 'y', 'z', 'h', 's', 'sdg', 't', 'tdg', 'cx', 'cy', 'cz', 'ch', 'cs', 'csdg', 'ct', 'ctdg'],
    'gates':[],
    'simulator': True,
    'local': True,
    'conditional': True,
    'max_shots': 10000,
    'memory': True,
    'description': 'Usual AER simulator.',
    'url': 'https://github.com/Qiskit/qiskit-aer',
    'coupling_map': None,
    'configuration': {
        'max_circuits': 100,
        'max_experiments': 100,
        'memory_slots': 32,
        'n_registers': 32,
    }
}


In [29]:
configuration = qpus[0].backend.__dict__
configuration

{'basis_gates': '',
 'conditional': True,
 'custom_instructions': '',
 'description': 'Usual AER simulator.',
 'is_simulator': True,
 'max_shots': 10000,
 'memory': True,
 'n_qubits': 32,
 'name': 'BasicAer',
 'simulator': 'AerSimulator',
 'url': 'https://github.com/Qiskit/qiskit-aer',
 'version': '0.0.1'}

In [41]:
args = {
        "backend_name": configuration["name"],
        "backend_version": configuration["version"],
        "n_qubits": configuration["n_qubits"],
        "basis_gates": ['u1', 'u2', 'u3', 'u', 'p', 'r', 'rx', 'ry', 'rz', 'id', 'x', 'y', 'z', 'h', 's', 'sdg', 'sx', 'sxdg', 't', 'tdg', 'swap', 'cx', 'cy', 'cz', 'csx', 'cp', 'cu', 'cu1', 'cu2', 'cu3', 'rxx', 'ryy', 'rzz', 'rzx', 'ccx', 'ccz', 'crx', 'cry', 'crz', 'cswap', 'mcx', 'mcy', 'mcz', 'mcsx', 'mcp', 'mcphase', 'mcu', 'mcu1', 'mcu2', 'mcu3', 'mcrx', 'mcry', 'mcrz', 'mcr', 'mcswap', 'unitary', 'diagonal', 'multiplexer', 'pauli', 'mcx_gray', 'ecr', 'store'],
        "gates":[],# TODO: comprobar que esto no peta y que funciona adecuadamente
        "local":False,
        "simulator":configuration["is_simulator"],
        "conditional":configuration["conditional"],
        "open_pulse":False,# TODO: lo ponemos así porque Aer no soporta OpenPulse, habría que modificarlo en caso de usar un backend que lo soporte.
        "memory":configuration["memory"],
        "max_shots":configuration["max_shots"],
        "coupling_map":[]
    }

In [42]:
from qiskit.providers.models import BackendConfiguration


/scratch/93357/ipykernel_1386693/882571660.py:1: DeprecationWarning: qiskit.providers.models is deprecated since Qiskit 1.2 and will be removed in Qiskit 2.0. With the removal of Qobj, there is no need for these schema-conformant objects. If you still need to use them, it could be because you are using a BackendV1, which is also deprecated in favor of BackendV2.
  from qiskit.providers.models import BackendConfiguration


In [43]:
target = convert_to_target(BackendConfiguration(**args))

/opt/cesga/qmio/hpc/software/Compiler/gcc/12.3.0/qiskit/1.2.4-python-3.9.9/lib/python3.9/site-packages/qiskit/providers/backend_compat.py:126: RuntimeWarning: No gate definition for mcp can be found and is being excluded from the generated target. You can use `custom_name_mapping` to provide a definition for this operation.
  warnings.warn(
/opt/cesga/qmio/hpc/software/Compiler/gcc/12.3.0/qiskit/1.2.4-python-3.9.9/lib/python3.9/site-packages/qiskit/providers/backend_compat.py:126: RuntimeWarning: No gate definition for mcx can be found and is being excluded from the generated target. You can use `custom_name_mapping` to provide a definition for this operation.
  warnings.warn(
/opt/cesga/qmio/hpc/software/Compiler/gcc/12.3.0/qiskit/1.2.4-python-3.9.9/lib/python3.9/site-packages/qiskit/providers/backend_compat.py:126: RuntimeWarning: No gate definition for mcy can be found and is being excluded from the generated target. You can use `custom_name_mapping` to provide a definition for this

In [44]:
pm = generate_preset_pass_manager(optimization_level = 1, target = target)

In [45]:
pm.run(qc).draw()

2025-01-23 15:47:58 - Pass: ContainsInstruction - 0.00834 (ms)
2025-01-23 15:47:58 - Pass: InverseCancellation - 0.07939 (ms)
2025-01-23 15:47:58 - Pass: UnitarySynthesis - 0.00644 (ms)
2025-01-23 15:47:58 - Pass: HighLevelSynthesis - 0.13542 (ms)
2025-01-23 15:47:58 - Pass: BasisTranslator - 0.08726 (ms)
2025-01-23 15:47:58 - Pass: Depth - 0.04888 (ms)
2025-01-23 15:47:58 - Pass: FixedPoint - 0.00882 (ms)
2025-01-23 15:47:58 - Pass: Size - 0.02527 (ms)
2025-01-23 15:47:58 - Pass: FixedPoint - 0.00548 (ms)
2025-01-23 15:47:58 - Pass: Optimize1qGatesDecomposition - 0.41366 (ms)
2025-01-23 15:47:58 - Pass: InverseCancellation - 0.05102 (ms)
2025-01-23 15:47:58 - Pass: GatesInBasis - 0.07963 (ms)
2025-01-23 15:47:58 - Pass: Depth - 0.04172 (ms)
2025-01-23 15:47:58 - Pass: FixedPoint - 0.00715 (ms)
2025-01-23 15:47:58 - Pass: Size - 0.02265 (ms)
2025-01-23 15:47:58 - Pass: FixedPoint - 0.00548 (ms)
2025-01-23 15:47:58 - Pass: Optimize1qGatesDecomposition - 0.32139 (ms)
2025-01-23 15:47:58 

┌─────────┐         ░ ┌─┐               
  q1_0: ──┤ U2(0,0) ├─────────░─┤M├───────────────
          └──┬───┬──┘  ┌─┐    ░ └╥┘┌─┐            
  q1_1: ─────┤ H ├─────┤M├────░──╫─┤M├────────────
        ┌────┴───┴────┐└╥┘    ░  ║ └╥┘┌─┐         
  q2_0: ┤ U2(-π/2,-π) ├─╫─────░──╫──╫─┤M├─────────
        └────┬───┬────┘ ║     ░  ║  ║ └╥┘┌─┐      
  q2_1: ─────┤ H ├──────╫─────░──╫──╫──╫─┤M├──────
             ├───┤      ║ ┌─┐ ░  ║  ║  ║ └╥┘┌─┐   
  q2_2: ─────┤ H ├──────╫─┤M├─░──╫──╫──╫──╫─┤M├───
             ├───┤      ║ └╥┘ ░  ║  ║  ║  ║ └╥┘┌─┐
  q2_3: ─────┤ H ├──────╫──╫──░──╫──╫──╫──╫──╫─┤M├
             └───┘      ║  ║  ░  ║  ║  ║  ║  ║ └╥┘
  c1: 2/════════════════╩══╬═════╬══╬══╬══╬══╬══╬═
                        1  ║     ║  ║  ║  ║  ║  ║ 
  c2: 4/═══════════════════╩═════╬══╬══╬══╬══╬══╬═
                           2     ║  ║  ║  ║  ║  ║ 
meas: 6/═════════════════════════╩══╩══╩══╩══╩══╩═
                                 0  1  2  3  4  5

In [39]:
lista = ["u1", "u2", "u3", "u", "p", "r", "rx", "ry", "rz", "id", "x", "y", "z", "h", "s",
            "sdg",
            "sx",
            "sxdg",
            "t",
            "tdg",
            "swap",
            "cx",
            "cy",
            "cz",
            "csx",
            "cp",
            "cu",
            "cu1",
            "cu2",
            "cu3",
            "rxx",
            "ryy",
            "rzz",
            "rzx",
            "ccx",
            "ccz",
            "crx",
            "cry",
            "crz",
            "cswap",
            "mcx",
            "mcy",
            "mcz",
            "mcsx",
            "mcp",
            "mcphase",
            "mcu",
            "mcu1",
            "mcu2",
            "mcu3",
            "mcrx",
            "mcry",
            "mcrz",
            "mcr",
            "mcswap",
            "unitary",
            "diagonal",
            "multiplexer",
            "pauli",
            "mcx_gray",
            "ecr",
            "store"]

In [40]:
print(lista)

['u1', 'u2', 'u3', 'u', 'p', 'r', 'rx', 'ry', 'rz', 'id', 'x', 'y', 'z', 'h', 's', 'sdg', 'sx', 'sxdg', 't', 'tdg', 'swap', 'cx', 'cy', 'cz', 'csx', 'cp', 'cu', 'cu1', 'cu2', 'cu3', 'rxx', 'ryy', 'rzz', 'rzx', 'ccx', 'ccz', 'crx', 'cry', 'crz', 'cswap', 'mcx', 'mcy', 'mcz', 'mcsx', 'mcp', 'mcphase', 'mcu', 'mcu1', 'mcu2', 'mcu3', 'mcrx', 'mcry', 'mcrz', 'mcr', 'mcswap', 'unitary', 'diagonal', 'multiplexer', 'pauli', 'mcx_gray', 'ecr', 'store']


In [22]:
from qiskit.providers.models import BackendConfiguration
backend_config = BackendConfiguration(
    backend_name = 'BasicAer',
    backend_version = '0.0.1',
    description = 'Usual AER simulator.',
    n_qubits = 32,
    basis_gates = ['u1', 'u2', 'u3', 'x', 'y', 'z', 'h', 's', 'sdg', 't', 'tdg', 'cx', 'cy', 'cz', 'ch', 'cs', 'csdg', 'ct', 'ctdg'],
    gates = [],
    max_shots = 10000,
    coupling_map = [],
    local = False,
    simulator = True,
    conditional = True,
    open_pulse = False,
    memory = True
)
'''

Más argumentos que toma la clase BackendConfiguration:

        supported_instructions=None,
        dynamic_reprate_enabled=False,
        rep_delay_range=None,
        default_rep_delay=None,
        max_experiments=None,
        sample_name=None,
        n_registers=None,
        register_map=None,
        configurable=None,
        credits_required=None,
        online_date=None,
        display_name=None,
        description=None,
        tags=None,
        dt=None,
        dtm=None,
        processor_type=None,
        parametric_pulses=None,
        **kwargs,

'''

/scratch/93357/ipykernel_1386693/753950192.py:1: DeprecationWarning: qiskit.providers.models is deprecated since Qiskit 1.2 and will be removed in Qiskit 2.0. With the removal of Qobj, there is no need for these schema-conformant objects. If you still need to use them, it could be because you are using a BackendV1, which is also deprecated in favor of BackendV2.
  from qiskit.providers.models import BackendConfiguration


'\n\nMás argumentos que toma la clase BackendConfiguration:\n\n        supported_instructions=None,\n        dynamic_reprate_enabled=False,\n        rep_delay_range=None,\n        default_rep_delay=None,\n        max_experiments=None,\n        sample_name=None,\n        n_registers=None,\n        register_map=None,\n        configurable=None,\n        credits_required=None,\n        online_date=None,\n        display_name=None,\n        description=None,\n        tags=None,\n        dt=None,\n        dtm=None,\n        processor_type=None,\n        parametric_pulses=None,\n        **kwargs,\n\n'

In [23]:
from qiskit.providers.backend_compat import convert_to_target
from qiskit.transpiler import Target

In [24]:
target_from_backendconfig = convert_to_target(backend_config)

/opt/cesga/qmio/hpc/software/Compiler/gcc/12.3.0/qiskit/1.2.4-python-3.9.9/lib/python3.9/site-packages/qiskit/providers/backend_compat.py:126: RuntimeWarning: No gate definition for ct can be found and is being excluded from the generated target. You can use `custom_name_mapping` to provide a definition for this operation.
  warnings.warn(
/opt/cesga/qmio/hpc/software/Compiler/gcc/12.3.0/qiskit/1.2.4-python-3.9.9/lib/python3.9/site-packages/qiskit/providers/backend_compat.py:126: RuntimeWarning: No gate definition for ctdg can be found and is being excluded from the generated target. You can use `custom_name_mapping` to provide a definition for this operation.
  warnings.warn(


In [25]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

In [26]:
 SPM = generate_preset_pass_manager(optimization_level = 1, target = target_from_backendconfig)

In [28]:
SPM.run(qc).draw()

2025-01-23 15:10:17 - Pass: ContainsInstruction - 0.00668 (ms)
2025-01-23 15:10:17 - Pass: InverseCancellation - 0.07200 (ms)
2025-01-23 15:10:17 - Pass: UnitarySynthesis - 0.00620 (ms)
2025-01-23 15:10:17 - Pass: HighLevelSynthesis - 0.14901 (ms)
2025-01-23 15:10:17 - Pass: BasisTranslator - 0.08011 (ms)
2025-01-23 15:10:17 - Pass: Depth - 0.04578 (ms)
2025-01-23 15:10:17 - Pass: FixedPoint - 0.00811 (ms)
2025-01-23 15:10:17 - Pass: Size - 0.02265 (ms)
2025-01-23 15:10:17 - Pass: FixedPoint - 0.00525 (ms)
2025-01-23 15:10:18 - Pass: Optimize1qGatesDecomposition - 0.24080 (ms)
2025-01-23 15:10:18 - Pass: InverseCancellation - 0.08512 (ms)
2025-01-23 15:10:18 - Pass: GatesInBasis - 0.06294 (ms)
2025-01-23 15:10:18 - Pass: Depth - 0.04196 (ms)
2025-01-23 15:10:18 - Pass: FixedPoint - 0.00763 (ms)
2025-01-23 15:10:18 - Pass: Size - 0.02265 (ms)
2025-01-23 15:10:18 - Pass: FixedPoint - 0.00620 (ms)
2025-01-23 15:10:18 - Pass: Optimize1qGatesDecomposition - 0.21076 (ms)
2025-01-23 15:10:18 

┌─────────┐         ░ ┌─┐               
  q1_0: ──┤ U2(0,0) ├─────────░─┤M├───────────────
          └──┬───┬──┘  ┌─┐    ░ └╥┘┌─┐            
  q1_1: ─────┤ H ├─────┤M├────░──╫─┤M├────────────
        ┌────┴───┴────┐└╥┘    ░  ║ └╥┘┌─┐         
  q2_0: ┤ U2(-π/2,-π) ├─╫─────░──╫──╫─┤M├─────────
        └────┬───┬────┘ ║     ░  ║  ║ └╥┘┌─┐      
  q2_1: ─────┤ H ├──────╫─────░──╫──╫──╫─┤M├──────
             ├───┤      ║ ┌─┐ ░  ║  ║  ║ └╥┘┌─┐   
  q2_2: ─────┤ H ├──────╫─┤M├─░──╫──╫──╫──╫─┤M├───
             ├───┤      ║ └╥┘ ░  ║  ║  ║  ║ └╥┘┌─┐
  q2_3: ─────┤ H ├──────╫──╫──░──╫──╫──╫──╫──╫─┤M├
             └───┘      ║  ║  ░  ║  ║  ║  ║  ║ └╥┘
  c1: 2/════════════════╩══╬═════╬══╬══╬══╬══╬══╬═
                        1  ║     ║  ║  ║  ║  ║  ║ 
  c2: 4/═══════════════════╩═════╬══╬══╬══╬══╬══╬═
                           2     ║  ║  ║  ║  ║  ║ 
meas: 6/═════════════════════════╩══╩══╩══╩══╩══╩═
                                 0  1  2  3  4  5

In [159]:
json_data

{'instructions': [{'name': 'h', 'qubits': [0], 'params': []},
  {'name': 'h', 'qubits': [1], 'params': []},
  {'name': 'h', 'qubits': [2], 'params': []},
  {'name': 'h', 'qubits': [3], 'params': []},
  {'name': 'h', 'qubits': [4], 'params': []},
  {'name': 'cx', 'qubits': [0, 1], 'params': []},
  {'name': 'cx', 'qubits': [2, 3], 'params': []},
  {'name': 'cx', 'qubits': [0, 4], 'params': []},
  {'name': 'measure', 'qubits': [0], 'memory': [1]},
  {'name': 'measure', 'qubits': [1], 'memory': [7]},
  {'name': 'h', 'qubits': [0], 'params': []},
  {'name': 'h', 'qubits': [1], 'params': []},
  {'name': 'h', 'qubits': [2], 'params': []},
  {'name': 'h', 'qubits': [3], 'params': []},
  {'name': 'h', 'qubits': [4], 'params': []},
  {'name': 'measure', 'qubits': [0], 'memory': [9]},
  {'name': 'measure', 'qubits': [1], 'memory': [10]},
  {'name': 'measure', 'qubits': [2], 'memory': [11]},
  {'name': 'measure', 'qubits': [3], 'memory': [12]},
  {'name': 'measure', 'qubits': [4], 'memory': [13]}]

In [172]:
if all([k in circuit.keys() for k in ['instructions', 'num_qubits', 'num_clbits', 'classical_registers']]):
    instructions = circuit['instructions']
    num_qubits = circuit['num_qubits']
    num_clbits = circuit['num_clbits']
    classical_registers = circuit['classical_registers']
else:
    raise KeyError("Circuit dict must have 'instructions', 'num_qbits' and 'num_clbits'.")

In [224]:
qc = QuantumCircuit(num_qubits)

regs = [QuantumRegister(num_qubits, 'q')]
for cr, lista in classical_registers.items():
    qc.add_register(ClassicalRegister(len(lista), cr))

qc.data

[]

In [193]:
from qiskit.circuit import QuantumRegister, ClassicalRegister, CircuitInstruction, Operation, Instruction

In [200]:
inst = Operation(name = 'h', num_qubits = 1, num_clbits = 0, params = [])

TypeError: Operation() takes no arguments

In [206]:
from qiskit.circuit import CircuitInstruction, Instruction, Qubit, QuantumRegister
c = CircuitInstruction(operation=, qubits=(Qubit(QuantumRegister(2, 'q'), 0),), clbits=())

In [209]:
Instruction(name='h', num_qubits=1, num_clbits=0, params=[])

Instruction(name='h', num_qubits=1, num_clbits=0, params=[])

In [210]:
json_data

{'instructions': [{'name': 'h', 'qubits': [0], 'params': []},
  {'name': 'h', 'qubits': [1], 'params': []},
  {'name': 'h', 'qubits': [2], 'params': []},
  {'name': 'h', 'qubits': [3], 'params': []},
  {'name': 'h', 'qubits': [4], 'params': []},
  {'name': 'cx', 'qubits': [0, 1], 'params': []},
  {'name': 'cx', 'qubits': [2, 3], 'params': []},
  {'name': 'cx', 'qubits': [0, 4], 'params': []},
  {'name': 'measure', 'qubits': [0], 'memory': [1]},
  {'name': 'measure', 'qubits': [1], 'memory': [7]},
  {'name': 'h', 'qubits': [0], 'params': []},
  {'name': 'h', 'qubits': [1], 'params': []},
  {'name': 'h', 'qubits': [2], 'params': []},
  {'name': 'h', 'qubits': [3], 'params': []},
  {'name': 'h', 'qubits': [4], 'params': []},
  {'name': 'measure', 'qubits': [0], 'memory': [9]},
  {'name': 'measure', 'qubits': [1], 'memory': [10]},
  {'name': 'measure', 'qubits': [2], 'memory': [11]},
  {'name': 'measure', 'qubits': [3], 'memory': [12]},
  {'name': 'measure', 'qubits': [4], 'memory': [13]}]

In [211]:
qc.data

[CircuitInstruction(operation=Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0),), clbits=()), CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)), clbits=())]

In [217]:
instructions = json_data['instructions']
json_data

{'instructions': [{'name': 'h', 'qubits': [0], 'params': []},
  {'name': 'h', 'qubits': [1], 'params': []},
  {'name': 'h', 'qubits': [2], 'params': []},
  {'name': 'h', 'qubits': [3], 'params': []},
  {'name': 'h', 'qubits': [4], 'params': []},
  {'name': 'cx', 'qubits': [0, 1], 'params': []},
  {'name': 'cx', 'qubits': [2, 3], 'params': []},
  {'name': 'cx', 'qubits': [0, 4], 'params': []},
  {'name': 'measure', 'qubits': [0], 'memory': [1]},
  {'name': 'measure', 'qubits': [1], 'memory': [7]},
  {'name': 'h', 'qubits': [0], 'params': []},
  {'name': 'h', 'qubits': [1], 'params': []},
  {'name': 'h', 'qubits': [2], 'params': []},
  {'name': 'h', 'qubits': [3], 'params': []},
  {'name': 'h', 'qubits': [4], 'params': []},
  {'name': 'measure', 'qubits': [0], 'memory': [9]},
  {'name': 'measure', 'qubits': [1], 'memory': [10]},
  {'name': 'measure', 'qubits': [2], 'memory': [11]},
  {'name': 'measure', 'qubits': [3], 'memory': [12]},
  {'name': 'measure', 'qubits': [4], 'memory': [13]}]

In [237]:
from qiskit.circuit import QuantumRegister, ClassicalRegister, CircuitInstruction, Instruction, Qubit, Clbit
def from_json_to_qc(circuit_dict):
    if isinstance(circuit_dict, QuantumCircuit):
        raise TypeError("Circuit provided is already a <class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>.")
    elif isinstance(circuit_dict, dict):
        circuit = circuit_dict
    else:
        raise TypeError("Please provide circuit_dict as a dict object.")

    if all([k in circuit.keys() for k in ['instructions', 'num_qubits', 'num_clbits', 'classical_registers']]):
        instructions = circuit['instructions']
        num_qubits = circuit['num_qubits']
        num_clbits = circuit['num_clbits']
        classical_registers = circuit['classical_registers']
    else:
        raise KeyError("Circuit dict must have 'instructions', 'num_qbits' and 'num_clbits'.")
    
    qc = QuantumCircuit(num_qubits)

    for cr, lista in classical_registers.items():
        qc.add_register(ClassicalRegister(len(lista), cr))


    for instruction in instructions:
        if instruction['name'] != 'measure':
            inst = CircuitInstruction( 
                operation = Instruction(name = instruction['name'],
                                        num_qubits = len(instruction['qubits']),
                                        num_clbits = 0,
                                        params = instruction['params']
                                        ),
                qubits = (Qubit(QuantumRegister(num_qubits, 'q'), q) for q in instruction['qubits']),
                clbits = ()
                )
            qc.append(inst)
        elif instruction['name'] == 'measure':
            bit = instruction['memory'][0]
            for k,v in classical_registers.items():
                if bit in v:
                    reg = k
                    l = len(v)
                    clbit = v.index(bit)
            inst = CircuitInstruction( 
                operation = Instruction(name = instruction['name'],
                                        num_qubits = 1,
                                        num_clbits = 1,
                                        params = []
                                        ),
                qubits = (Qubit(QuantumRegister(num_qubits, 'q'), q) for q in instruction['qubits']),
                clbits = (Clbit(ClassicalRegister(l, reg), clbit),)
                )
            qc.append(inst)
    return qc

In [244]:
qcc = from_json_to_qc(json_data)
qcc.cregs

[ClassicalRegister(6, 'j'),
 ClassicalRegister(3, 'c'),
 ClassicalRegister(5, 'meas')]

In [260]:
from qiskit.circuit import QuantumRegister, ClassicalRegister, CircuitInstruction, Instruction, Qubit, Clbit

circuito = QuantumCircuit(QuantumRegister(8,'q1'), QuantumRegister(5, 'q2'), ClassicalRegister(5, 'c'), ClassicalRegister(5, 'j'))

circuito.h(0); circuito.x(8); circuito.measure(0, 7)

for d in circuito.data:
    print(d)
    print(' ')

CircuitInstruction(operation=Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(8, 'q1'), 0),), clbits=())
 
CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'q2'), 0),), clbits=())
 
CircuitInstruction(operation=Instruction(name='measure', num_qubits=1, num_clbits=1, params=[]), qubits=(Qubit(QuantumRegister(8, 'q1'), 0),), clbits=(Clbit(ClassicalRegister(5, 'j'), 2),))
 


In [269]:
[r._register.name for r in qcc.data[0].qubits]

['q']

In [273]:
def qc_to_json(qc):
    """
    Transforms a QuantumCircuit to json.

    Args:
    ---------
    qc (QuantumCircuit): circuit to transform to json.

    Return:
    ---------
    json dict with the circuit information.
    """
    quantum_registers = {}
    for qr in qc.qregs:
        quantum_registers[qr.name] = qr.size

    countsq = []

    valuesq = list(quantum_registers.values())

    for i, v in enumerate(valuesq):
        if i == 0:
            countsq.append(list(range(0, v)))
        else:
            countsq.append(list(range(sum(valuesq[:i]), sum(valuesq[:i])+v)))

    for i,k in enumerate(quantum_registers.keys()):
        quantum_registers[k] = countsq[i]


    classical_registers = {}
    for cr in qc.cregs:
        classical_registers[cr.name] = cr.size

    counts = []

    values = list(classical_registers.values())

    for i, v in enumerate(values):
        if i == 0:
            counts.append(list(range(0, v)))
        else:
            counts.append(list(range(sum(values[:i]), sum(values[:i])+v)))

    for i,k in enumerate(classical_registers.keys()):
        classical_registers[k] = counts[i]
    
    json_data = {
        "instructions":[],
        "num_qubits":sum([q.size for q in qc.qregs]),
        "num_clbits": sum([c.size for c in qc.cregs]),
        "quantum_registers":quantum_registers,
        "classical_registers":classical_registers
    }
    for i in range(len(qc.data)):
        if qc.data[i].name == "barrier":
            pass
        elif qc.data[i].name != "measure":

            qreg = [r._register.name for r in qc.data[i].qubits]
            qubit = [q._index for q in qc.data[i].qubits]

            json_data["instructions"].append({"name":qc.data[i].name, 
                                              "qubits":[quantum_registers[k][q] for k,q in zip(qreg,qubit)],
                                              "params":qc.data[i].params
                                             })
        else:
            qreg = [r._register.name for r in qc.data[i].qubits]
            qubit = [q._index for q in qc.data[i].qubits]
            
            creg = [r._register.name for r in qc.data[i].clbits]
            bit = [b._index for b in qc.data[i].clbits]

            json_data["instructions"].append({"name":qc.data[i].name,
                                              "qubits":[quantum_registers[k][q] for k,q in zip(qreg,qubit)],
                                              "memory":[classical_registers[k][b] for k,b in zip(creg,bit)]
                                             })
                

    return json_data

In [282]:
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumRegister, ClassicalRegister, CircuitInstruction, Instruction, Qubit, Clbit


def from_json_to_qc(circuit_dict):
    if isinstance(circuit_dict, QuantumCircuit):
        raise TypeError("Circuit provided is already a <class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>.")
    elif isinstance(circuit_dict, dict):
        circuit = circuit_dict
    else:
        raise TypeError("Please provide circuit_dict as a dict object.")

    if all([k in circuit.keys() for k in ['instructions', 'num_qubits', 'num_clbits', 'classical_registers']]):
        instructions = circuit['instructions']
        num_qubits = circuit['num_qubits']
        num_clbits = circuit['num_clbits']
        classical_registers = circuit['classical_registers']
    else:
        raise KeyError("Circuit dict must have 'instructions', 'num_qbits' and 'num_clbits'.")
    
    qc = QuantumCircuit(num_qubits)

    for cr, lista in classical_registers.items():
        qc.add_register(ClassicalRegister(len(lista), cr))


    for instruction in instructions:
        if instruction['name'] != 'measure':
            inst = CircuitInstruction( 
                operation = Instruction(name = instruction['name'],
                                        num_qubits = len(instruction['qubits']),
                                        num_clbits = 0,
                                        params = instruction['params']
                                        ),
                qubits = (Qubit(QuantumRegister(num_qubits, 'q'), q) for q in instruction['qubits']),
                clbits = ()
                )
            qc.append(inst)
        elif instruction['name'] == 'measure':
            bit = instruction['memory'][0]
            for k,v in classical_registers.items():
                if bit in v:
                    reg = k
                    l = len(v)
                    clbit = v.index(bit)
            inst = CircuitInstruction( 
                operation = Instruction(name = instruction['name'],
                                        num_qubits = 1,
                                        num_clbits = 1,
                                        params = []
                                        ),
                qubits = (Qubit(QuantumRegister(num_qubits, 'q'), q) for q in instruction['qubits']),
                clbits = (Clbit(ClassicalRegister(l, reg), clbit),)
                )
            qc.append(inst)
    return qc

In [4]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
qc = QuantumCircuit(
    QuantumRegister(2, 'q1'), QuantumRegister(4, 'q2'),
    ClassicalRegister(2, 'c1'), ClassicalRegister(4, 'c2')
    )

qc.h(range(qc.num_qubits))

qc.x(0); qc.measure(1,1)
qc.sdg(2); qc.measure(4,4)


qc.measure_all()

qc.draw()

len(qc.data)

print(type(qc))

<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>


In [295]:
json_data = qc_to_json(qc)
json_data

{'instructions': [{'name': 'h', 'qubits': [0], 'params': []},
  {'name': 'h', 'qubits': [1], 'params': []},
  {'name': 'h', 'qubits': [2], 'params': []},
  {'name': 'h', 'qubits': [3], 'params': []},
  {'name': 'h', 'qubits': [4], 'params': []},
  {'name': 'h', 'qubits': [5], 'params': []},
  {'name': 'x', 'qubits': [0], 'params': []},
  {'name': 'measure', 'qubits': [1], 'memory': [1]},
  {'name': 'sdg', 'qubits': [2], 'params': []},
  {'name': 'measure', 'qubits': [4], 'memory': [4]},
  {'name': 'measure', 'qubits': [0], 'memory': [6]},
  {'name': 'measure', 'qubits': [1], 'memory': [7]},
  {'name': 'measure', 'qubits': [2], 'memory': [8]},
  {'name': 'measure', 'qubits': [3], 'memory': [9]},
  {'name': 'measure', 'qubits': [4], 'memory': [10]},
  {'name': 'measure', 'qubits': [5], 'memory': [11]}],
 'num_qubits': 6,
 'num_clbits': 12,
 'quantum_registers': {'q1': [0, 1], 'q2': [2, 3, 4, 5]},
 'classical_registers': {'c1': [0, 1],
  'c2': [2, 3, 4, 5],
  'meas': [6, 7, 8, 9, 10, 11]}

In [297]:
len(from_json_to_qc(json_data).data)

16

In [296]:
for d in from_json_to_qc(json_data).data:
    print(d)
    print(" ")

CircuitInstruction(operation=Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(6, 'q'), 0),), clbits=())
 
CircuitInstruction(operation=Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(6, 'q'), 1),), clbits=())
 
CircuitInstruction(operation=Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(6, 'q'), 2),), clbits=())
 
CircuitInstruction(operation=Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(6, 'q'), 3),), clbits=())
 
CircuitInstruction(operation=Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(6, 'q'), 4),), clbits=())
 
CircuitInstruction(operation=Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(6, 'q'), 5),), clbits=())
 
CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(Q

In [21]:
from qiskit.providers.models import BackendConfiguration

/scratch/96406/ipykernel_3641570/3066083476.py:1: DeprecationWarning: qiskit.providers.models is deprecated since Qiskit 1.2 and will be removed in Qiskit 2.0. With the removal of Qobj, there is no need for these schema-conformant objects. If you still need to use them, it could be because you are using a BackendV1, which is also deprecated in favor of BackendV2.
  from qiskit.providers.models import BackendConfiguration


In [33]:
from backend import Backend
from qiskit import QuantumCircuit
from circuit import from_json_to_qc
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.providers.backend_compat import convert_to_target


def transpiler(circuit, backend, opt_level = 1):
    
    # transformo el circuito a QuantumCircuit
    if isinstance(circuit, QuantumCircuit):
        circuit = circuit
    elif isinstance(circuit, dict):
        try:
            circuit = from_json_to_qc(circuit)
        except:
            raise KeyError("Circuit json not correct, requiered keys must be: 'instructions', 'num_qubits', 'num_clbits', 'quantum_resgisters' and 'classical_registers'.")
    else:
        raise TypeError("Circuit must be <class 'qiskit.circuit.quantumcircuit.QuantumCircuit'> or dict.")
    

    # Limpio bien el diccionario de configuración para generar el target apropiado

    if isinstance(backend, Backend):
        configuration = backend.__dict__
    else:
        raise TypeError("backend must be <class 'python.backend.Backend'>")
    
    
    
    args = {
        "backend_name": configuration["name"],
        "backend_version": configuration["version"],
        "n_qubits": configuration["n_qubits"],
        "basis_gates": configuration["basis_gates"],
        "gates":[],# TODO: comprobar que esto no peta y que funciona adecuadamente
        "local":False,
        "simulator":configuration["is_simulator"],
        "conditional":configuration["conditional"],
        "open_pulse":False,# TODO: lo ponemos así porque Aer no soporta OpenPulse, habría que modificarlo en caso de usar un backend que lo soporte.
        "memory":configuration["memory"],
        "max_shots":configuration["max_shots"],
        "coupling_map":configuration["coupling_map"]
    }

    backend_configuration = BackendConfiguration(**args)
    
    target =  convert_to_target(backend_configuration)
    pm = generate_preset_pass_manager(optimization_level = opt_level, target = target)

    return pm.run(circuit)

In [34]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(10,10); qc.h(range(10)); qc.cx(0,1);qc.cx(1,2);qc.cx(5,6);qc.cx(8,9)
qc.measure(0,0); qc.h(range(10)); qc.cx(4,3);qc.cx(3,8);qc.cx(2,6);qc.cx(0,7);
qc.measure(3,3); qc.measure(4,4); qc.h(range(10)); qc.measure_all()

In [41]:
from backend import Backend
configuration = qpus[0].backend.__dict__
configuration['basis_gates'] = ['u1', 'u2', 'u3', 'u', 'p', 'r', 'rx', 'ry', 'rz', 'id', 'x', 'y', 'z', 'h', 's', 'sdg', 'sx', 'sxdg', 't', 'tdg', 'swap', 'cx', 'cy', 'cz', 'csx', 'cp', 'cu', 'cu1', 'cu3', 'rxx', 'ryy', 'rzz', 'rzx', 'ccx', 'ccz', 'crx', 'cry', 'crz', 'cswap','ecr']
configuration['coupling_map'] =[]
backend = Backend(configuration)
backend.info()

--- Backend configuration ---
basis_gates: ['u1', 'u2', 'u3', 'u', 'p', 'r', 'rx', 'ry', 'rz', 'id', 'x', 'y', 'z', 'h', 's', 'sdg', 'sx', 'sxdg', 't', 'tdg', 'swap', 'cx', 'cy', 'cz', 'csx', 'cp', 'cu', 'cu1', 'cu3', 'rxx', 'ryy', 'rzz', 'rzx', 'ccx', 'ccz', 'crx', 'cry', 'crz', 'cswap', 'ecr']
conditional: True
custom_instructions: 
description: Usual AER simulator.
is_simulator: True
max_shots: 10000
memory: True
n_qubits: 32
name: BasicAer
simulator: AerSimulator
url: https://github.com/Qiskit/qiskit-aer
version: 0.0.1
coupling_map: []


In [42]:
print(qc)

         ┌───┐          ┌─┐┌───┐                    ┌───┐      ░ ┌─┐         »
    q_0: ┤ H ├──■───────┤M├┤ H ├─────────────────■──┤ H ├──────░─┤M├─────────»
         ├───┤┌─┴─┐     └╥┘├───┤┌───┐            │  └───┘      ░ └╥┘┌─┐      »
    q_1: ┤ H ├┤ X ├──■───╫─┤ H ├┤ H ├────────────┼─────────────░──╫─┤M├──────»
         ├───┤└───┘┌─┴─┐ ║ ├───┤└───┘            │  ┌───┐      ░  ║ └╥┘┌─┐   »
    q_2: ┤ H ├─────┤ X ├─╫─┤ H ├────────────■────┼──┤ H ├──────░──╫──╫─┤M├───»
         ├───┤┌───┐├───┤ ║ └───┘            │    │  └┬─┬┘┌───┐ ░  ║  ║ └╥┘┌─┐»
    q_3: ┤ H ├┤ H ├┤ X ├─╫───■──────────────┼────┼───┤M├─┤ H ├─░──╫──╫──╫─┤M├»
         ├───┤├───┤└─┬─┘ ║   │   ┌─┐ ┌───┐  │    │   └╥┘ └───┘ ░  ║  ║  ║ └╥┘»
    q_4: ┤ H ├┤ H ├──■───╫───┼───┤M├─┤ H ├──┼────┼────╫────────░──╫──╫──╫──╫─»
         ├───┤└───┘┌───┐ ║   │   └╥┘ ├───┤  │    │    ║        ░  ║  ║  ║  ║ »
    q_5: ┤ H ├──■──┤ H ├─╫───┼────╫──┤ H ├──┼────┼────╫────────░──╫──╫──╫──╫─»
         ├───┤┌─┴─┐├───┤ ║   │    ║  └───┘┌─┴─┐  │  

In [43]:
qct = transpiler( qc, backend )

2025-01-28 16:10:55 - Pass: ContainsInstruction - 0.00811 (ms)
2025-01-28 16:10:55 - Pass: InverseCancellation - 0.40650 (ms)
2025-01-28 16:10:55 - Pass: UnitarySynthesis - 0.00620 (ms)
2025-01-28 16:10:55 - Pass: HighLevelSynthesis - 0.27490 (ms)
2025-01-28 16:10:55 - Pass: BasisTranslator - 0.18740 (ms)
2025-01-28 16:10:55 - Pass: Depth - 0.07701 (ms)
2025-01-28 16:10:55 - Pass: FixedPoint - 0.00834 (ms)
2025-01-28 16:10:55 - Pass: Size - 0.03171 (ms)
2025-01-28 16:10:55 - Pass: FixedPoint - 0.00572 (ms)
2025-01-28 16:10:55 - Pass: Optimize1qGatesDecomposition - 0.76842 (ms)
2025-01-28 16:10:55 - Pass: InverseCancellation - 0.17333 (ms)
2025-01-28 16:10:55 - Pass: GatesInBasis - 0.14281 (ms)
2025-01-28 16:10:55 - Pass: Depth - 0.06247 (ms)
2025-01-28 16:10:55 - Pass: FixedPoint - 0.00596 (ms)
2025-01-28 16:10:55 - Pass: Size - 0.02956 (ms)
2025-01-28 16:10:55 - Pass: FixedPoint - 0.00501 (ms)
2025-01-28 16:10:55 - Pass: ContainsInstruction - 0.00620 (ms)


/scratch/96406/ipykernel_3641570/4001562844.py:1: DeprecationWarning: The class ``qiskit.providers.models.backendconfiguration.BackendConfiguration`` is deprecated as of qiskit 1.2. It will be removed in the 2.0 release. The models in ``qiskit.providers.models`` are part of the deprecated `BackendV1` workflow and no longer necessary for `BackendV2`. If a user workflow requires these representations it likely relies on deprecated functionality and should be updated to use `BackendV2`.
  qct = transpiler( qc, backend )


In [40]:
print(qct)

         ┌───┐               ┌─┐┌───┐          ┌───┐      ░ ┌─┐               »
    q_0: ┤ H ├──■────────────┤M├┤ H ├───────■──┤ H ├──────░─┤M├───────────────»
         ├───┤┌─┴─┐          └╥┘└───┘       │  └───┘      ░ └╥┘┌─┐            »
    q_1: ┤ H ├┤ X ├──■────────╫─────────────┼─────────────░──╫─┤M├────────────»
         ├───┤└───┘┌─┴─┐┌───┐ ║             │  ┌───┐      ░  ║ └╥┘┌─┐         »
    q_2: ┤ H ├─────┤ X ├┤ H ├─╫────────■────┼──┤ H ├──────░──╫──╫─┤M├─────────»
         ├───┤     └───┘└───┘ ║        │    │  └┬─┬┘┌───┐ ░  ║  ║ └╥┘┌─┐      »
    q_3: ┤ X ├────────────────╫───■────┼────┼───┤M├─┤ H ├─░──╫──╫──╫─┤M├──────»
         └─┬─┘      ┌─┐ ┌───┐ ║   │    │    │   └╥┘ └───┘ ░  ║  ║  ║ └╥┘┌─┐   »
    q_4: ──■────────┤M├─┤ H ├─╫───┼────┼────┼────╫────────░──╫──╫──╫──╫─┤M├───»
         ┌───┐      └╥┘ └───┘ ║   │    │    │    ║        ░  ║  ║  ║  ║ └╥┘┌─┐»
    q_5: ┤ H ├──■────╫────────╫───┼────┼────┼────╫────────░──╫──╫──╫──╫──╫─┤M├»
         ├───┤┌─┴─┐  ║  ┌───┐ ║   │  ┌─┴

In [12]:
from qiskit import QuantumCircuit

In [13]:
qc = QuantumCircuit(2); qc.h(0); qc.cx(0,1); qc.measure_all()

In [14]:
qpus

[<qpu.QPU at 0x1475f97faca0>, <qpu.QPU at 0x1475f97d9220>]

In [15]:
result = qpus[0].run(qc, shots = 4000, transpile = True).result()

2025-01-29 13:06:51 - Pass: ContainsInstruction - 0.00811 (ms)
2025-01-29 13:06:51 - Pass: InverseCancellation - 0.02241 (ms)
2025-01-29 13:06:51 - Pass: UnitarySynthesis - 0.00596 (ms)
2025-01-29 13:06:51 - Pass: HighLevelSynthesis - 0.08106 (ms)
2025-01-29 13:06:51 - Pass: BasisTranslator - 0.05198 (ms)
2025-01-29 13:06:51 - Pass: Depth - 0.03481 (ms)
2025-01-29 13:06:51 - Pass: FixedPoint - 0.00834 (ms)
2025-01-29 13:06:51 - Pass: Size - 0.01717 (ms)
2025-01-29 13:06:51 - Pass: FixedPoint - 0.00596 (ms)
2025-01-29 13:06:51 - Pass: Optimize1qGatesDecomposition - 0.14067 (ms)
2025-01-29 13:06:51 - Pass: InverseCancellation - 0.01383 (ms)
2025-01-29 13:06:51 - Pass: GatesInBasis - 0.03505 (ms)
2025-01-29 13:06:51 - Pass: Depth - 0.02432 (ms)
2025-01-29 13:06:51 - Pass: FixedPoint - 0.00620 (ms)
2025-01-29 13:06:51 - Pass: Size - 0.01359 (ms)
2025-01-29 13:06:51 - Pass: FixedPoint - 0.00501 (ms)
2025-01-29 13:06:51 - Pass: ContainsInstruction - 0.00644 (ms)
/mnt/netapp1/Store_CESGA/home

KeyboardInterrupt: 